In [1]:
# N_TRAIN_ROW_TOTAL = 101230332
# https://www.kaggle.com/spacelx/2020-r3id-clustering-question-tags/ (Version 5)
# Decide whether or not running on Kaggle
import os
KAGGLE_RUN = True
if os.path.isdir('/home/ubuntu'):
    KAGGLE_RUN = False
    
if KAGGLE_RUN:
    model_name = 'modelloop3/modelloop3.txt'
else:
    model_name = 'modelloop3.txt'

import sys
import pandas as pd
import numpy as np
import gc
from sklearn.metrics import roc_auc_score
from collections import defaultdict
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import datatable as dt

if not KAGGLE_RUN:
    sys.path.insert(0, './input')
    import local_work
    
import riiideducation

# Boto3 Setup and Download Files
if not KAGGLE_RUN:
    local_work = local_work.local_work()
    local_work.download_riiid_files()

import random
import os

# Random seed
SEED = 0
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

In [2]:
# Funcion for user stats with loops
def add_features(df, features_dicts, KAGGLE_RUN, is_inference):
    # -----------------------------------------------------------------------
    # Get feature dicts
    d_answered_correctly_u_count = features_dicts['d_answered_correctly_u_count']
    d_answered_correctly_u_sum = features_dicts['d_answered_correctly_u_sum']
    d_answered_correctly_prev5_u = features_dicts['d_answered_correctly_prev5_u']
    d_elapsed_time_u_sum = features_dicts['d_elapsed_time_u_sum']
    d_explanation_u_sum = features_dicts['d_explanation_u_sum']
    d_answered_correctly_q_count = features_dicts['d_answered_correctly_q_count']
    d_answered_correctly_q_sum = features_dicts['d_answered_correctly_q_sum']
    d_elapsed_time_q_sum = features_dicts['d_elapsed_time_q_sum']
    d_explanation_q_sum = features_dicts['d_explanation_q_sum']
    d_answered_correctly_uq = features_dicts['d_answered_correctly_uq']
    d_timestamp_u = features_dicts['d_timestamp_u']
    d_timestamp_u_incorrect = features_dicts['d_timestamp_u_incorrect']
    d_answered_correctly_up_sum = features_dicts['d_answered_correctly_up_sum']
    d_answered_correctly_up_count = features_dicts['d_answered_correctly_up_count']
    d_answered_correctly_uc_count = features_dicts['d_answered_correctly_uc_count']
    d_answered_correctly_uc_sum = features_dicts['d_answered_correctly_uc_sum']
    
    
    # -----------------------------------------------------------------------
    # Client features
    answered_correctly_u_avg = np.zeros(len(df), dtype = np.float32)
    answered_correctly_prev5_u_avg = np.zeros(len(df), dtype = np.float32)
    elapsed_time_u_avg = np.zeros(len(df), dtype = np.float32)
    explanation_u_avg = np.zeros(len(df), dtype = np.float32)
    timestamp_u_recency_1 = np.zeros(len(df), dtype = np.float32)
    timestamp_u_recency_2 = np.zeros(len(df), dtype = np.float32)
    timestamp_u_recency_3 = np.zeros(len(df), dtype = np.float32)
    timestamp_u_incorrect_recency = np.zeros(len(df), dtype = np.float32)
    # -----------------------------------------------------------------------
    # Question features
    answered_correctly_q_avg = np.zeros(len(df), dtype = np.float32)
    elapsed_time_q_avg = np.zeros(len(df), dtype = np.float32)
    explanation_q_avg = np.zeros(len(df), dtype = np.float32)
    # -----------------------------------------------------------------------
    # User Question
    answered_correctly_uq_count = np.zeros(len(df), dtype = np.int32)
    # -----------------------------------------------------------------------
    # User Part
    answered_correctly_up_count = np.zeros(len(df), dtype = np.int32)
    answered_correctly_up_avg = np.zeros(len(df), dtype = np.float32)
    # -----------------------------------------------------------------------
    # User Community
    answered_correctly_uc_count = np.zeros(len(df), dtype = np.int32)
    answered_correctly_uc_avg = np.zeros(len(df), dtype = np.float32)    
    # -----------------------------------------------------------------------
    
    DF_NROW = df.shape[0]
    for num, row in enumerate(df[['user_id'
                                  ,'answered_correctly'
                                  ,'content_id'
                                  ,'prior_question_elapsed_time'
                                  ,'prior_question_had_explanation'
                                  ,'timestamp'
                                  ,'part'
                                  ,'community']].values):
        
        if DF_NROW > 1000000 and not KAGGLE_RUN:
            if num % 500000 == 0:
                print(round(num/DF_NROW, 3))
                
        # Client features assignation
        # ------------------------------------------------------------------
        if d_answered_correctly_u_count[row[0]] != 0:
            answered_correctly_u_avg[num] = d_answered_correctly_u_sum[row[0]] / d_answered_correctly_u_count[row[0]]
            elapsed_time_u_avg[num] = d_elapsed_time_u_sum[row[0]] / d_answered_correctly_u_count[row[0]]
            explanation_u_avg[num] = d_explanation_u_sum[row[0]] / d_answered_correctly_u_count[row[0]]
        else:
            answered_correctly_u_avg[num] = np.nan
            elapsed_time_u_avg[num] = np.nan
            explanation_u_avg[num] = np.nan
            
        if len(d_timestamp_u[row[0]]) == 0:
            timestamp_u_recency_1[num] = np.nan
            timestamp_u_recency_2[num] = np.nan
            timestamp_u_recency_3[num] = np.nan
        elif len(d_timestamp_u[row[0]]) == 1:
            timestamp_u_recency_1[num] = row[5] - d_timestamp_u[row[0]][0]
            timestamp_u_recency_2[num] = np.nan
            timestamp_u_recency_3[num] = np.nan
        elif len(d_timestamp_u[row[0]]) == 2:
            timestamp_u_recency_1[num] = row[5] - d_timestamp_u[row[0]][1]
            timestamp_u_recency_2[num] = row[5] - d_timestamp_u[row[0]][0]
            timestamp_u_recency_3[num] = np.nan
        elif len(d_timestamp_u[row[0]]) == 3:
            timestamp_u_recency_1[num] = row[5] - d_timestamp_u[row[0]][2]
            timestamp_u_recency_2[num] = row[5] - d_timestamp_u[row[0]][1]
            timestamp_u_recency_3[num] = row[5] - d_timestamp_u[row[0]][0]
        
        if len(d_timestamp_u_incorrect[row[0]]) == 0:
            timestamp_u_incorrect_recency[num] = np.nan
        else:
            timestamp_u_incorrect_recency[num] = row[5] - d_timestamp_u_incorrect[row[0]][0]
            
        if len(d_answered_correctly_prev5_u[row[0]]) == 0:
            answered_correctly_prev5_u_avg[num] = np.nan
        else:
            answered_correctly_prev5_u_avg[num] = sum(d_answered_correctly_prev5_u[row[0]])/\
                                                    len(d_answered_correctly_prev5_u[row[0]])
            
        # ------------------------------------------------------------------
        # Question features assignation
        if d_answered_correctly_q_count[row[2]] != 0:
            answered_correctly_q_avg[num] = d_answered_correctly_q_sum[row[2]] / d_answered_correctly_q_count[row[2]]
            elapsed_time_q_avg[num] = d_elapsed_time_q_sum[row[2]] / d_answered_correctly_q_count[row[2]]
            explanation_q_avg[num] = d_explanation_q_sum[row[2]] / d_answered_correctly_q_count[row[2]]
        else:
            answered_correctly_q_avg[num] = np.nan
            elapsed_time_q_avg[num] = np.nan
            explanation_q_avg[num] = np.nan
            
        # ------------------------------------------------------------------
        # Client Question assignation
        answered_correctly_uq_count[num] = d_answered_correctly_uq[row[0]][row[2]]
        
        # ------------------------------------------------------------------
        # Client Part assignation
        answered_correctly_up_count[num] = d_answered_correctly_up_count[row[0]][row[6]]
        if d_answered_correctly_up_count[row[0]][row[6]] == 0:
            answered_correctly_up_avg[num] = np.nan
        else:
            answered_correctly_up_avg[num] = d_answered_correctly_up_sum[row[0]][row[6]] /\
                                                d_answered_correctly_up_count[row[0]][row[6]]
        # ------------------------------------------------------------------
        # Client Community assignation
        answered_correctly_uc_count[num] = d_answered_correctly_uc_count[row[0]][row[7]]
        if d_answered_correctly_uc_count[row[0]][row[7]] == 0:
            answered_correctly_uc_avg[num] = np.nan
        else:
            answered_correctly_uc_avg[num] = d_answered_correctly_uc_sum[row[0]][row[7]] /\
                                                d_answered_correctly_uc_count[row[0]][row[7]]            
        
        # ------------------------------------------------------------------
        # ------------------------------------------------------------------
        # Client features updates
        d_answered_correctly_u_count[row[0]] += 1
        d_elapsed_time_u_sum[row[0]] += row[3]
        d_explanation_u_sum[row[0]] += int(row[4])
        if len(d_timestamp_u[row[0]]) == 3:
            d_timestamp_u[row[0]].pop(0)
            d_timestamp_u[row[0]].append(row[5])
        else:
            d_timestamp_u[row[0]].append(row[5])
        # ------------------------------------------------------------------
        # Question features updates
        d_answered_correctly_q_count[row[2]] += 1
        d_elapsed_time_q_sum[row[2]] += row[3]
        d_explanation_q_sum[row[2]] += int(row[4])
        # ------------------------------------------------------------------
        # Client Question updates
        d_answered_correctly_uq[row[0]][row[2]] += 1
        # ------------------------------------------------------------------
        # Client Part updates
        d_answered_correctly_up_count[row[0]][row[6]] += 1 
        # ------------------------------------------------------------------
        # Client Community updates
        d_answered_correctly_uc_count[row[0]][row[7]] += 1      
        
        # ------------------------------------------------------------------
        # Flag for training and inference. Note: this is almost identical to update_features()
        if not is_inference:
            # ------------------------------------------------------------------
            # Client features updates
            d_answered_correctly_u_sum[row[0]] += row[1]
            if row[1] == 0:
                if len(d_timestamp_u_incorrect[row[0]]) == 1:
                    d_timestamp_u_incorrect[row[0]].pop(0)
                d_timestamp_u_incorrect[row[0]].append(row[5])
                
            if len(d_answered_correctly_prev5_u[row[0]]) == 5:
                d_answered_correctly_prev5_u[row[0]].pop(0)
            d_answered_correctly_prev5_u[row[0]].append(row[1])                
            
            # ------------------------------------------------------------------
            # Question features updates
            d_answered_correctly_q_sum[row[2]] += row[1]
            
            # ------------------------------------------------------------------
            # Client Part features updates
            d_answered_correctly_up_sum[row[0]][row[6]] += row[1]
            
            # ------------------------------------------------------------------
            # Client Community features updates
            d_answered_correctly_uc_sum[row[0]][row[7]] += row[1]
            
            # ------------------------------------------------------------------
             
            
    user_df = pd.DataFrame({'answered_correctly_u_avg': answered_correctly_u_avg
                            ,'elapsed_time_u_avg': elapsed_time_u_avg
                            ,'explanation_u_avg': explanation_u_avg
                            ,'answered_correctly_q_avg': answered_correctly_q_avg
                            ,'answered_correctly_prev5_u_avg': answered_correctly_prev5_u_avg
                            ,'elapsed_time_q_avg': elapsed_time_q_avg
                            ,'explanation_q_avg': explanation_q_avg
                            ,'answered_correctly_uq_count': answered_correctly_uq_count
                            ,'timestamp_u_recency_1': timestamp_u_recency_1
                            ,'timestamp_u_recency_2': timestamp_u_recency_2
                            ,'timestamp_u_recency_3': timestamp_u_recency_3
                            ,'timestamp_u_incorrect_recency': timestamp_u_incorrect_recency
                            ,'answered_correctly_up_avg' : answered_correctly_up_avg
                            ,'answered_correctly_up_count' : answered_correctly_up_count
                            ,'answered_correctly_uc_avg' : answered_correctly_uc_avg
                            ,'answered_correctly_uc_count' : answered_correctly_uc_count
                           })
    
    # Only need to concat rows if building a model or inferring
    if (not KAGGLE_RUN) or is_inference:
        df = pd.concat([df, user_df], axis = 1)
    return df
        
def update_features(df, features_dicts):
    # -----------------------------------------------------------------------
    # Get feature dicts
    d_answered_correctly_u_sum = features_dicts['d_answered_correctly_u_sum']
    d_answered_correctly_q_sum = features_dicts['d_answered_correctly_q_sum']
    d_timestamp_u_incorrect = features_dicts['d_timestamp_u_incorrect']
    d_answered_correctly_prev5_u = features_dicts['d_answered_correctly_prev5_u']
    d_answered_correctly_up_sum = features_dicts['d_answered_correctly_up_sum']
    d_answered_correctly_uc_sum = features_dicts['d_answered_correctly_uc_sum']
    
    for row in df[['user_id'
                   ,'answered_correctly'
                   ,'content_id'
                   ,'content_type_id'
                   ,'timestamp'
                   ,'part'
                   ,'community'
                  ]].values:
        if row[3] == 0:
            # ------------------------------------------------------------------
            # Client features updates
            d_answered_correctly_u_sum[row[0]] += row[1]
            if row[1] == 0:
                if len(d_timestamp_u_incorrect[row[0]]) == 1:
                    d_timestamp_u_incorrect[row[0]].pop(0)
                d_timestamp_u_incorrect[row[0]].append(row[4])
                
            if len(d_answered_correctly_prev5_u[row[0]]) == 5:
                d_answered_correctly_prev5_u[row[0]].pop(0)
            d_answered_correctly_prev5_u[row[0]].append(row[1])                   
            # ------------------------------------------------------------------
            # Question features updates
            d_answered_correctly_q_sum[row[2]] += row[1]
            
            # ------------------------------------------------------------------
            # Client Part features updates
            d_answered_correctly_up_sum[row[0]][row[5]] += row[1]
            
            # ------------------------------------------------------------------
            # Client Community features updates
            d_answered_correctly_uc_sum[row[0]][row[6]] += row[1]

def read_and_preprocess(KAGGLE_RUN):

    # Make feature_dicts dictionary for easy passing between functions
    features_dicts = {
        # Client dictionaries
        'd_answered_correctly_u_count': defaultdict(int)
        ,'d_answered_correctly_u_sum': defaultdict(int)
        ,'d_answered_correctly_prev5_u': defaultdict(list)
        ,'d_elapsed_time_u_sum': defaultdict(int)
        ,'d_explanation_u_sum': defaultdict(int)
        ,'d_timestamp_u': defaultdict(list)
        ,'d_timestamp_u_incorrect': defaultdict(list)
        
        # Question Dictionaries
        ,'d_answered_correctly_q_count': defaultdict(int)
        ,'d_answered_correctly_q_sum': defaultdict(int)
        ,'d_elapsed_time_q_sum': defaultdict(int)
        ,'d_explanation_q_sum': defaultdict(int)
        
        # Client Question dictionary (how many times has the client seen the question)
        ,'d_answered_correctly_uq': defaultdict(lambda: defaultdict(int))

    }
    
    # Load questions
    if not KAGGLE_RUN:
        questions_df = local_work.get_questions_data()
        question_cmnts = pd.read_csv('./input/question_cmnts_v5.csv')
        question_cmnts.columns = ['question_id', 'community']
    else:   
        questions_df = pd.read_csv('../input/riiid-test-answer-prediction/questions.csv')
        
    tmp = questions_df.shape[0]
    questions_df = questions_df.merge(question_cmnts, on = 'question_id', how = 'left')
    assert(tmp == questions_df.shape[0])
        
    # Change questions dtypes
    questions_df['part'] = questions_df['part'].astype(np.int32)
    questions_df['bundle_id'] = questions_df['bundle_id'].astype(np.int32)
    questions_df['community'] = questions_df['community'].astype(np.int8)
        
    # Need to load all columns when batching
    train_cols = ['row_id', 'timestamp', 'user_id', 'content_id', 'content_type_id',
               'task_container_id', 'user_answer', 'answered_correctly',
               'prior_question_elapsed_time', 'prior_question_had_explanation']
    
    # Columns we need
    feld_needed = ['timestamp'
                   ,'user_id'
                   ,'answered_correctly'
                   ,'content_id'
                   ,'content_type_id'
                   ,'prior_question_elapsed_time'
                   ,'prior_question_had_explanation'
                  ]
    total_train_count = 0
    for i in range(11):
        if not KAGGLE_RUN:
            N_TRAIN_ROWS = 1 * 1000000
            train = local_work.get_train_data(feld_needed, nrow=N_TRAIN_ROWS)
        else:
            train = dt.fread('../input/riiid-test-answer-prediction/train.csv'
                            ,skip_to_line = i * 10000000 + 2
                            ,max_nrows=10000000).to_pandas()
            train.columns = train_cols
            train = train[feld_needed]
        
        total_train_count += train.shape[0]
    
        # Filter by content_type_id to discard lectures
        train = train.loc[train.content_type_id == 0].reset_index(drop = True)


        # Changing dtype to avoid lightgbm error
        train['prior_question_had_explanation'] = train.prior_question_had_explanation.fillna(False).astype('int8')

        # Merge with questions dataframe
        train = pd.merge(train, questions_df[['question_id', 'part', 'community']]
                         ,left_on = 'content_id'
                         ,right_on = 'question_id'
                         ,how = 'left')

        if not KAGGLE_RUN:
            valid['prior_question_had_explanation'] = valid.prior_question_had_explanation.fillna(False).astype('int8')
            valid = pd.merge(valid, questions_df[['question_id', 'part', 'community']]
                             ,left_on = 'content_id'
                             ,right_on = 'question_id'
                             ,how = 'left')

        print('Train Feature Calculation Started - ' + str(i))
        train = add_features(train, features_dicts, KAGGLE_RUN = KAGGLE_RUN, is_inference = False)
        
        # Only run once for model buildling
        if not KAGGLE_RUN:
            break
    
    print('Number of training rows read: ' + str(total_train_count))
    
    # Free up space
    if KAGGLE_RUN:
        del train
        train = None
    
    # Add features for validation set
    if not KAGGLE_RUN:
        print('Valid Feature Calculation Started')
        valid = add_features(valid, features_dicts, KAGGLE_RUN = KAGGLE_RUN, is_inference = False)
    gc.collect()
    
    return train, valid, questions_df, features_dicts

# Function for training and evaluation
def train_and_evaluate(train, valid, KAGGLE_RUN, model_name):
    
    TARGET = 'answered_correctly'
    FEATURES = ['prior_question_elapsed_time'
                ,'prior_question_had_explanation'
                ,'part'
                ,'community'
                ,'answered_correctly_u_avg'
                ,'answered_correctly_prev5_u_avg'
                ,'elapsed_time_u_avg'
                ,'explanation_u_avg'
                ,'answered_correctly_q_avg'
                ,'elapsed_time_q_avg'
                ,'explanation_q_avg'
                ,'answered_correctly_uq_count'
                ,'timestamp_u_recency_1'
                ,'timestamp_u_recency_2'
                ,'timestamp_u_recency_3'
                ,'timestamp_u_incorrect_recency'
                ,'answered_correctly_up_avg'
                ,'answered_correctly_up_count'
                ,'answered_correctly_uc_avg'
                ,'answered_correctly_uc_count'
               ]
    
    if not KAGGLE_RUN:
        print(f'Traning with {train.shape[0]} rows and {len(FEATURES)} features')    
        d_train = lgb.Dataset(train[FEATURES], label=train[TARGET])
        d_valid = lgb.Dataset(valid[FEATURES], label=valid[TARGET])

        params = {'objective': 'binary', 
                  'seed': SEED,
                  'metric': 'auc',
                  'learning_rate': .1,
                  'num_leaves': 120,
                  'feature_fraction': .8,
                  'bagging_fraction': .8,
                 }

        if True:
            model = lgb.train(
                            params = params,
                            train_set = d_train,
                            num_boost_round = 10000,
                            valid_sets = [d_train, d_valid],
                            early_stopping_rounds = 20,
                            verbose_eval = 50
                            )
            model.save_model(f'./output/' + model_name)
            lgb.plot_importance(model, importance_type='gain')
            lgb.plot_importance(model, importance_type='split')
        else:
            model = lgb.Booster(model_file='./output/' + model_name)
            lgb.plot_importance(model, importance_type='gain')
    else:
        model = lgb.Booster(model_file='/kaggle/input/' + model_name)

    return TARGET, FEATURES, model

# Using time series api that simulates production predictions
def inference(TARGET, FEATURES, model, questions_df, features_dicts):
    
    # Get api iterator and predictor
    env = riiideducation.make_env()
    iter_test = env.iter_test()
    set_predict = env.predict
    
    previous_test_df = None
    for (test_df, sample_prediction_df) in iter_test:
        if previous_test_df is not None:
            previous_test_df[TARGET] = eval(test_df["prior_group_answers_correct"].iloc[0])
            update_features(previous_test_df, features_dicts)
        previous_test_df = test_df.copy()
        test_df = test_df[test_df['content_type_id'] == 0].reset_index(drop = True)
        test_df['prior_question_had_explanation'] = test_df.prior_question_had_explanation.fillna(False).astype('int8')
        test_df = pd.merge(test_df, questions_df[['question_id', 'part', 'community']]
                           ,left_on = 'content_id'
                           ,right_on = 'question_id'
                           ,how = 'left')
        test_df[TARGET] = 0
        test_df = add_features(test_df, features_dicts, KAGGLE_RUN = KAGGLE_RUN, is_inference = True)
        test_df[TARGET] =  model.predict(test_df[FEATURES])
        set_predict(test_df[['row_id', TARGET]])
        
    print('Job Done')

In [ ]:
train[['user_id', 'answered_correctly', 'community', 'answered_correctly_uc_count', 'answered_correctly_uc_avg']][0:50]

In [3]:
train, valid, questions_df, features_dicts = read_and_preprocess(KAGGLE_RUN)

Joining validation sets
Validation size: 107607
Train size: 872486
Train Feature Calculation Started - 0
Number of training rows read: 1000000
Valid Feature Calculation Started


In [6]:
TARGET, FEATURES, model = train_and_evaluate(train, valid, KAGGLE_RUN, model_name)

Traning with 872486 rows and 20 features
[LightGBM] [Info] Number of positive: 571306, number of negative: 301180
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.020699 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3708
[LightGBM] [Info] Number of data points in the train set: 872486, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654802 -> initscore=0.640217
[LightGBM] [Info] Start training from score 0.640217
Training until validation scores don't improve for 20 rounds


KeyboardInterrupt: 

In [5]:
inference(TARGET, FEATURES, model, questions_df, features_dicts)

KeyError: "['community', 'part'] not in index"